# Getting started with ONNX IR 🌱
The ONNX IR ships with the ONNX Script package and is available as `onnxscript.ir`.
To create an IR object from ONNX file, load it as `ModelProto` and call
`ir.from_proto()`:

In [1]:
# Define an example model for this example
MODEL_TEXT = r"""
<
   ir_version: 8,
   opset_import: ["" : 18],
   producer_name: "pytorch",
   producer_version: "2.0.0"
>
torch_jit (float[5,5,5] input_0) => (float[5,5] val_19, float[5,5] val_6) {
   val_1 = Constant <value_int: ints = [1]> ()
   val_2 = Shape <start: int = 0> (val_1)
   val_3 = Size (val_2)
   val_4 = Constant <value: tensor = int64 {0}> ()
   val_5 = Equal (val_3, val_4)
   val_6 = ReduceMean <keepdims: int = 0, noop_with_empty_axes: int = 0> (input_0, val_1)
   val_7 = ReduceMean <keepdims: int = 1, noop_with_empty_axes: int = 0> (input_0, val_1)
   val_8 = Shape <start: int = 0> (input_0)
   val_9 = Gather <axis: int = 0> (val_8, val_1)
   val_10 = ReduceProd <keepdims: int = 0, noop_with_empty_axes: int = 0> (val_9)
   val_11 = Sub (input_0, val_7)
   val_12 = Mul (val_11, val_11)
   val_13 = ReduceMean <keepdims: int = 0, noop_with_empty_axes: int = 0> (val_12, val_1)
   val_14 = Cast <to: int = 1> (val_10)
   val_15 = Mul (val_13, val_14)
   val_16 = Constant <value: tensor = int64 {1}> ()
   val_17 = Sub (val_10, val_16)
   val_18 = Cast <to: int = 1> (val_17)
   val_19 = Div (val_15, val_18)
}
"""

In [2]:
import onnx

from onnxscript import ir

# Load the model as onnx.ModelProto
# You can also load the model from a file using onnx.load("model.onnx")
model_proto = onnx.parser.parse_model(MODEL_TEXT)

# Create an IR object from the model
model = ir.from_proto(model_proto)

Now we can explore the IR object

In [3]:
print(f"The main graph has {len(model.graph)} nodes.")

The main graph has 19 nodes.


All inputs

In [4]:
print(model.graph.inputs)

[Input('input_0', type=Tensor(FLOAT), shape=[5,5,5], producer=None, index=None)]


All outputs

In [5]:
print(model.graph.outputs)

[Value('val_19', type=Tensor(FLOAT), shape=[5,5], producer=, index=0), Value('val_6', type=Tensor(FLOAT), shape=[5,5], producer=, index=0)]


Nodes that uses the first input

In [6]:
print(list(model.graph.inputs[0].uses()))

[(Node(name='', domain='', op_type='ReduceMean', inputs=(Input('input_0', type=Tensor(FLOAT), shape=[5,5,5], producer=None, index=None), Value('val_1', type=None, shape=None, producer=, index=0)), attributes=OrderedDict([('keepdims', AttrInt64('keepdims', 0)), ('noop_with_empty_axes', AttrInt64('noop_with_empty_axes', 0))]), overload='', outputs=(Value('val_6', type=Tensor(FLOAT), shape=[5,5], producer=, index=0),), version=None, doc_string=None), 0), (Node(name='', domain='', op_type='ReduceMean', inputs=(Input('input_0', type=Tensor(FLOAT), shape=[5,5,5], producer=None, index=None), Value('val_1', type=None, shape=None, producer=, index=0)), attributes=OrderedDict([('keepdims', AttrInt64('keepdims', 1)), ('noop_with_empty_axes', AttrInt64('noop_with_empty_axes', 0))]), overload='', outputs=(Value('val_7', type=None, shape=None, producer=, index=0),), version=None, doc_string=None), 0), (Node(name='', domain='', op_type='Shape', inputs=(Input('input_0', type=Tensor(FLOAT), shape=[5,5,

The node that produces the last output (as the i-th output)

In [7]:
print(model.graph.outputs[-1].producer())
print(model.graph.outputs[-1].index())

%"val_6"<FLOAT,[5,5]> ⬅️ ::ReduceMean(%"input_0", %"val_1") {keepdims=0, noop_with_empty_axes=0}
0


Print the graph

In [9]:
model.graph.display(
    page=False
)  # Set page=True to use a pager in the terminal so long outputs are scrollable

graph(
    name=torch_jit,
    inputs=(
        %"input_0"<FLOAT,[5,5,5]>
    ),
    outputs=(
        %"val_19"<FLOAT,[5,5]>,
        %"val_6"<FLOAT,[5,5]>
    ),
) {
     0 |  # :anonymous_node:128897555281104
          %"val_1"<?,?> ⬅️ ::Constant() {value_int=[1]}
     1 |  # :anonymous_node:128897554321872
          %"val_2"<?,?> ⬅️ ::Shape(%"val_1") {start=0}
     2 |  # :anonymous_node:128895578494032
          %"val_3"<?,?> ⬅️ ::Size(%"val_2")
     3 |  # :anonymous_node:128895578494352
          %"val_4"<?,?> ⬅️ ::Constant() {value=TensorProtoTensor<INT64,[]>(name='')}
     4 |  # :anonymous_node:128895578494512
          %"val_5"<?,?> ⬅️ ::Equal(%"val_3", %"val_4")
     5 |  # :anonymous_node:128895578494992
          %"val_6"<FLOAT,[5,5]> ⬅️ ::ReduceMean(%"input_0", %"val_1") {keepdims=0, noop_with_empty_axes=0}
     6 |  # :anonymous_node:128895578495312
          %"val_7"<?,?> ⬅️ ::ReduceMean(%"input_0", %"val_1") {keepdims=1, noop_with_empty_axes=0}
     7 |  # :anonymous_node:128895578495472
          %"val_8"<?,?> ⬅️ ::Shape(%"input_0") {start=0}
     8 |  # :anonymous_node:128895578495632
          %"val_9"<?,?> ⬅️ ::Gather(%"val_8", %"val_1") {axis=0}
     9 |  # :anonymous_node:128895578495952
          %"val_10"<?,?> ⬅️ ::ReduceProd(%"val_9") {keepdims=0, noop_with_empty_axes=0}
    10 |  # :anonymous_node:128895578496272
          %"val_11"<?,?> ⬅️ ::Sub(%"input_0", %"val_7")
    11 |  # :anonymous_node:128895578496592
          %"val_12"<?,?> ⬅️ ::Mul(%"val_11", %"val_11")
    12 |  # :anonymous_node:128895578497072
          %"val_13"<?,?> ⬅️ ::ReduceMean(%"val_12", %"val_1") {keepdims=0, noop_with_empty_axes=0}
    13 |  # :anonymous_node:128895578497712
          %"val_14"<?,?> ⬅️ ::Cast(%"val_10") {to=1}
    14 |  # :anonymous_node:128895578498192
          %"val_15"<?,?> ⬅️ ::Mul(%"val_13", %"val_14")
    15 |  # :anonymous_node:128895578498672
          %"val_16"<?,?> ⬅️ ::Constant() {value=TensorProtoTensor<INT64,[]>(name='')}
    16 |  # :anonymous_node:128895578498832
          %"val_17"<?,?> ⬅️ ::Sub(%"val_10", %"val_16")
    17 |  # :anonymous_node:128895578499152
          %"val_18"<?,?> ⬅️ ::Cast(%"val_17") {to=1}
    18 |  # :anonymous_node:128895578499632
          %"val_19"<FLOAT,[5,5]> ⬅️ ::Div(%"val_15", %"val_18")
    return %"val_19"<FLOAT,[5,5]>, %"val_6"<FLOAT,[5,5]>
}

Convert from the IR object back to ModelProto

In [10]:
model_proto_back = ir.to_proto(model)

## Next steps

Read the introductions for a more detailed introduction of the IR
(Documentation in progress 🚧)